https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews

https://amazon-reviews-2023.github.io/

# Citation
Bridging Language and Items for Retrieval and Recommendation
Yupeng Hou, Jiacheng Li, Zhankui He, An Yan, Xiusi Chen, Julian McAuley
arXiv
[pdf](https://urldefense.com/v3/__https://arxiv.org/pdf/2403.03952.pdf__;!!Mih3wA!EGkx27nmvdVMhh2uxQ7Mc0rNrXQwV8GsOpd3uSc6ZjJGAVhgy9o5bn3Jeb73P4Lz7oL2dIDMdZR4IHI$)

https://scikit-learn.org/0.15/modules/scaling_strategies.html

In [14]:
!pip install pymongo
!pip install pandas
!pip install bs4
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -------- ------------------------------- 2.6/12.8 MB 15.9 MB/s eta 0:00:01
     ---------------------- ----------------- 7.1/12.8 MB 18.4 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 19.7 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 18.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
from MongoDB import MongoDB
import pandas as pd


def load_amazon_data(mongo_db:MongoDB, amazon_reviews_db:MongoDB, category:str, url:str):
    if category not in mongo_db:
        print(category, mongo_db)
        df = pd.read_json(
            path_or_buf = url, 
            compression = "gzip",
            lines =
            True
        )
        mongo_db.append(
            **{
                "collection_name" : category,
                "df" : df
            } | (
                {
                    "timeseries" : {
                        "timeField" : "timestamp"
                    }
                } if mongo_db == amazon_reviews_db else {}
            )
        )

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from MongoDB import MongoDB
from multiprocessing import Process

amazon_reviews_db = MongoDB("Amazon_Reviews")
amazon_items_db = MongoDB("Amazon_Items")
response = requests.get("https://amazon-reviews-2023.github.io/")
soup  = BeautifulSoup(response.text, "html.parser")
file_links = soup.find_all(
    name = "a",
    attrs = {
        "href" : re.compile(r'.*https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/.*')
    }
)
for file_link in file_links[:10]:
    categories = file_link.attrs["href"].split("/")[-1].split(".jsonl.gz")[0].partition("meta_")
    mongo_db = amazon_reviews_db if categories[-1] == "" else amazon_items_db
    category = categories[0] if categories[-1] == "" else categories[-1]
    Process(
        target = load_amazon_data, 
        args = (
            mongo_db, 
            amazon_reviews_db, 
            category, 
            file_link.attrs["href"],
        )
    ).start()

In [12]:
from typing import Generator
import pandas as pd
import pymongo


class MongoDB:
    def __init__(self, database_name:str, url:str = "mongodb://localhost:27017/"):
        self.__database_name = database_name
        self.__url = url
        
        
    def append(self, df:pd.DataFrame, collection_name:str, timeseries:dict = None) -> bool:
        if not timeseries:
            timeseries = {}
            
        else:
            timeseries = {
                "timeseries" : timeseries
            }
            
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            if collection_name not in db.list_collection_names():
                db.create_collection(**{"name" : collection_name} | timeseries)
                
            db[collection_name].insert_many(
                df.to_dict(
                    orient = "records"
                )
            )
            
            
    def count(self, collection_name:str, query:dict = {}) -> int:
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            return db[collection_name].count_documents(query)
        
        
    def list_collection_names(self):
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            return db.list_collection_names()
        
        
    def query(self, collection_name:str, query:dict = {}):
        rows = []
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            cursor  = db[collection_name].find(
                filter = query
            )
            previous_retrieved = 0
            for row in cursor:
                if len(row) > 0 and cursor.retrieved != previous_retrieved:
                    previous_retrieved = cursor.retrieved
                    yield pd.DataFrame(rows)
                    rows.clear()
                        
                rows.append(row)
                
            yield pd.DataFrame(rows)
            
            
    def aggregate(self, collection_name:str, pipeline:list):
        rows = []
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            cursor  = db[collection_name].aggregate(pipeline)
            return pd.DataFrame(
                [
                    row
                    for row 
                    in cursor
                ]
            )
    
    
    def __contains__(self, collection_name):
        with pymongo.MongoClient(self.__url) as client:
            db = client[self.__database_name]
            return collection_name in db.list_collection_names()
        
        
    def __repr__(self):
        return self.__database_name

In [1]:
from MongoDB import MongoDB
import pandas as pd


amazon_reviews_db = MongoDB("Amazon_Reviews")
amazon_items_db = MongoDB("Amazon_Items")

KeyboardInterrupt: 

In [ ]:
# for collection_name in amazon_reviews_db.list_collection_names():.count(collection_name)
#     print(collection_name, amazon_reviews_db.count(collection_name))

Pet_Supplies 360000
system.buckets.Pet_Supplies 201892
Cell_Phones_and_Accessories 260000
system.buckets.Cell_Phones_and_Accessories 159883
Electronics 360000
system.buckets.Electronics 206652
CDs_and_Vinyl 400000
system.buckets.CDs_and_Vinyl 251001
Baby_Products 320000
system.buckets.Baby_Products 189706
Amazon_Fashion 2500939
system.buckets.Amazon_Fashion 884152
Toys_and_Games 360000
system.buckets.Toys_and_Games 198077
Magazine_Subscriptions 71497
system.buckets.Magazine_Subscriptions 58141
Grocery_and_Gourmet_Food 360000
system.buckets.Grocery_and_Gourmet_Food 199580
Industrial_and_Scientific 290000
system.buckets.Industrial_and_Scientific 173895
Movies_and_TV 410000
system.buckets.Movies_and_TV 222404
Patio_Lawn_and_Garden 250000
system.buckets.Patio_Lawn_and_Garden 150257
Software 370000
system.buckets.Software 212219
Gift_Cards 152410
system.buckets.Gift_Cards 101028
Books 270000
system.buckets.Books 170023
Clothing_Shoes_and_Jewelry 350000
system.buckets.Clothing_Shoes_and_Jewe

In [ ]:
results = """
Pet_Supplies 360000
system.buckets.Pet_Supplies 201892
Cell_Phones_and_Accessories 260000
system.buckets.Cell_Phones_and_Accessories 159883
Electronics 360000
system.buckets.Electronics 206652
CDs_and_Vinyl 400000
system.buckets.CDs_and_Vinyl 251001
Baby_Products 320000
system.buckets.Baby_Products 189706
Amazon_Fashion 2500939
system.buckets.Amazon_Fashion 884152
Toys_and_Games 360000
system.buckets.Toys_and_Games 198077
Magazine_Subscriptions 71497
system.buckets.Magazine_Subscriptions 58141
Grocery_and_Gourmet_Food 360000
system.buckets.Grocery_and_Gourmet_Food 199580
Industrial_and_Scientific 290000
system.buckets.Industrial_and_Scientific 173895
Movies_and_TV 410000
system.buckets.Movies_and_TV 222404
Patio_Lawn_and_Garden 250000
system.buckets.Patio_Lawn_and_Garden 150257
Software 370000
system.buckets.Software 212219
Gift_Cards 152410
system.buckets.Gift_Cards 101028
Books 270000
system.buckets.Books 170023
Clothing_Shoes_and_Jewelry 350000
system.buckets.Clothing_Shoes_and_Jewelry 187206
Health_and_Personal_Care 350000
system.buckets.Health_and_Personal_Care 228253
Musical_Instruments 310000
system.buckets.Musical_Instruments 188044
Kindle_Store 280000
system.buckets.Kindle_Store 168478
Handmade_Products 370000
system.buckets.Handmade_Products 220170
system.views 34
Home_and_Kitchen 340000
system.buckets.Home_and_Kitchen 185938
Automotive 290000
system.buckets.Automotive 169511
Subscription_Boxes 16216
system.buckets.Subscription_Boxes 13346
Health_and_Household 260000
system.buckets.Health_and_Household 149124
Tools_and_Home_Improvement 270000
system.buckets.Tools_and_Home_Improvement 159367
Sports_and_Outdoors 280000
system.buckets.Sports_and_Outdoors 166159
Office_Products 270000
system.buckets.Office_Products 161496
Video_Games 290000
system.buckets.Video_Games 184937
Beauty_and_Personal_Care 270000
system.buckets.Beauty_and_Personal_Care 152860
Unknown 330000
system.buckets.Unknown 190146
Arts_Crafts_and_Sewing 370000
system.buckets.Arts_Crafts_and_Sewing 202562
Appliances 340000
system.buckets.Appliances 210106
All_Beauty 701528
system.buckets.All_Beauty 278351
Digital_Music 130434
system.buckets.Digitcal_Musi 99288
""".split("\n")[1:-1]

In [23]:
cols = pd.DataFrame(
    [
        [col.split(" ")[0],float(col.split(" ")[1])]
        for col
        in results
        if not col.startswith("system")
    ]
)

In [35]:
amazon_reviews_db.aggregate(
    collection_name = "Magazine_Subscriptions",
    pipeline = [
        {
            "$group" : {
                "_id": None, 
                "Max Date": {
                    "$min": "$timestamp"
                } 
            }  
        }
    ]
)

,_id,Max Date
0,None,2001-10-26 19:41:06


In [29]:
cols[0].iloc[0]

'Pet_Supplies'

https://medium.com/@aleksej.gudkov/llama-cpp-python-examples-a-guide-to-using-llama-models-with-python-1df9ba7a5fcd

In [30]:
cols

,0,1
0,Pet_Supplies,360000.0
1,Cell_Phones_and_Accessories,260000.0
2,Electronics,360000.0
3,CDs_and_Vinyl,400000.0
4,Baby_Products,320000.0
5,Amazon_Fashion,2500939.0
6,Toys_and_Games,360000.0
7,Magazine_Subscriptions,71497.0
8,Grocery_and_Gourmet_Food,360000.0
9,Industrial_and_Scientific,290000.0


In [ ]:
import random
from spacy.lang.en.stop_words import STOP_WORDS as stopwords_list

def text_filter(a:pd.Series, label:pd.Series, exclude_lists:list):
    data = []
    for rev_id in a_dict.keys():
        tokens = []
        for token in a_dict.get(rev_id):
            if not token.text in exclude_lists:
                tokens.append(token.text)
        data.append((' '.join(tokens), label))
    return data

def prepare_data(pos_docs, neg_docs, exclude_lists):
    data = text_filter(pos_docs, 1, exclude_lists)
    data += text_filter(neg_docs, -1, exclude_lists)
    random.seed(42)
    random.shuffle(data)
    texts = []
    labels = []
    for item in data:
        texts.append(item[0])
        labels.append(item[1])
        
    return texts, labels


In [ ]:
from MongoDB import MongoDB
import pandas as pd
import en_core_web_sm

collection_name = "Amazon_Fashion"
nlp = en_core_web_sm.load()
amazon_reviews_db = MongoDB("Amazon_Reviews")
total_reviews = amazon_reviews_db.count(collection_name)

i = 0
for reviews in amazon_reviews_db.query(collection_name):
    print(len(reviews))
    reviews["sentiment"] = reviews["rating"].apply(
        lambda rating: 1 if rating > 3 else 0 if rating < 3 else -1
    )
    reviews["text"] = reviews["text"].apply(
        lambda text: nlp(
            text,
            disable = ["ner"]
        )
    )
    
    if i > 2:
        break
    
    i += 1

101
32403
38109
45113


: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
train_counts = count_vect.fit_transform(train_data) 

In [12]:
import string


punctuation_list = [
    punct 
    for punct 
    in string.punctuation
]

In [ ]:
texts, labels = prepare_data(pos_docs, neg_docs, punctuation_list)
print(len(texts), len(labels))
print(texts[0]) 

In [ ]:
from MongoDB import MongoDB
import pandas as pd


amazon_reviews_db = MongoDB("Amazon_Reviews")
amazon_items_db = MongoDB("Amazon_Items")


reviews = pd.concat(
    objs = [
        batch
        for batch
        in amazon_reviews_db.query("All_Beauty")
    ]
)
reviews["sentiment"] = reviews["rating"].apply(
    lambda rating: 1 if rating > 3 else 0 if rating < 3 else -1
)
items = pd.concat(
    objs = [
        batch
        for batch
        in amazon_items_db.query("All_Beauty")
    ]
)
pd.merge(
    left = reviews,
    right = items,
    on = "parent_asin",
    how = "left"
)

KeyboardInterrupt: 